# This is the fourth exercise from EPFL EE-559 – DEEP LEARNING course

## hello all!


The exercise itself is located  <a href="https://fleuret.org/ee559/materials/ee559-practical-4.pdf"> here</a><br>
For any questions please mail me at tomer@nahshon.net<br>
Some of the code here is taken straight from the <a href="https://fleuret.org/ee559/src/dlc_practical_4_solution.py">solution</a> itself but presented in a better visualized way.

In [1]:
import math
import os
import torch
import torchvision

In [14]:
from torch import nn
from torch.nn import functional as F
import dlc_practical_prologue as prologue

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        return x



def train_model(model, train_input, train_target, mini_batch_size):
    eta = 1e-1
    criterion = nn.MSELoss()
    for e in range(0, 25):
        sum_loss = 0
        # We do this with mini-batches
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            sum_loss = sum_loss + loss.item()
            model.zero_grad()
            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= eta * p.grad

        print(e, sum_loss)
    return model


In [15]:
train_input, train_target, test_input, test_target = prologue.load_data(one_hot_labels = True, normalize = True, flatten = False)

* Using MNIST
** Reduce the data-set (use --full for the full thing)
** Use 1000 train and 1000 test samples


In [16]:
model = Net()
mini_batch_size = 100

In [17]:
model = train_model(model, train_input, train_target, mini_batch_size)

0 0.9222614988684654
1 0.7986443713307381
2 0.7361812219023705
3 0.6806511580944061
4 0.6304269134998322
5 0.585875429213047
6 0.5485511235892773
7 0.5188374146819115
8 0.5031090676784515
9 0.4850390776991844
10 0.44415660202503204
11 0.4236020930111408
12 0.4210911951959133
13 0.41060032323002815
14 0.378169447183609
15 0.3901103623211384
16 0.35503595881164074
17 0.33910796977579594
18 0.3309978637844324
19 0.32291887514293194
20 0.3112474028021097
21 0.29939712956547737
22 0.2898037526756525
23 0.2875694092363119
24 0.2921291943639517


In [18]:
def compute_nb_errors(model, input, target, mini_batch_size):
    nb_errors = 0

    for b in range(0, input.size(0), mini_batch_size):
        output = model(input.narrow(0, b, mini_batch_size))
        _, predicted_classes = output.max(1)
        for k in range(mini_batch_size):
            if target[b + k, predicted_classes[k]] <= 0:
                nb_errors = nb_errors + 1

    return nb_errors

In [21]:
for k in range(10):
    model = Net()
    train_model(model, train_input, train_target, mini_batch_size)
    nb_test_errors = compute_nb_errors(model, test_input, test_target, mini_batch_size)
    print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),
                                                      nb_test_errors, test_input.size(0)))

0 0.9223078638315201
1 0.7994844764471054
2 0.7354758158326149
3 0.677043192088604
4 0.6261133626103401
5 0.584796380251646
6 0.5513072162866592
7 0.517948966473341
8 0.48677581548690796
9 0.47102760896086693
10 0.44857367500662804
11 0.4095415994524956
12 0.43664824590086937
13 0.38655906170606613
14 0.36051754653453827
15 0.35297261364758015
16 0.35233717039227486
17 0.338614659383893
18 0.3191841971129179
19 0.3187356647104025
20 0.31714397110044956
21 0.29652771167457104
22 0.29213669151067734
23 0.2777680493891239
24 0.27749719843268394
test error Net 14.10% 141/1000
0 0.9162067770957947
1 0.7941851243376732
2 0.7182531803846359
3 0.6506625711917877
4 0.5935915820300579
5 0.5465456880629063
6 0.5118890814483166
7 0.497282300144434
8 0.49226562678813934
9 0.43021615594625473
10 0.4478701129555702
11 0.39254794269800186
12 0.37645223736763
13 0.3685768321156502
14 0.3500896729528904
15 0.3349238522350788
16 0.32701030001044273
17 0.31545907631516457
18 0.29730983078479767
19 0.28909